In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

In [2]:
ori_data = np.load('../Data/samples_24/energy_norm_truth.npy')
fake_data = np.load('../Data/ldm_fake_energy_16.npy')
iterations = 5

In [3]:
## Context-FID Score
context_fid_score = []
for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.08250649120021053 

Iter 1:  context-fid = 0.08157033956399139 

Iter 2:  context-fid = 0.08969995247067984 

Iter 3:  context-fid = 0.0900558918535797 

Iter 4:  context-fid = 0.08780792779096153 

Final Score:  0.0863281205758846 ± 0.0049936157740787


In [4]:
## Correlational Score
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.897215784905162 

Iter 1:  cross-correlation = 1.6009859269388031 

Iter 2:  cross-correlation = 1.0548390658545395 

Iter 3:  cross-correlation = 1.4854154720173534 

Iter 4:  cross-correlation = 1.3295457609785015 

Final Score:  1.273600402138872 ± 0.3644969600221273


In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

In [6]:
### Discriminative score
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use tf.global_variables instead.


training: 100%|██████████| 2000/2000 [00:25<00:00, 77.99it/s]


Iter 0:  0.2227998985544002 , 0.6238904387522192 , 0.8217093583565813 



training: 100%|██████████| 2000/2000 [00:25<00:00, 78.11it/s]


...

Iter 2:  0.2622368754755262 , 0.7195029165609942 , 0.8049708343900583 



training: 100%|██████████| 2000/2000 [00:29<00:00, 68.82it/s]


Iter 3:  0.24942936850114128 , 0.758559472482881 , 0.7402992645194014 



training: 100%|██████████| 2000/2000 [00:40<00:00, 48.99it/s]


Iter 4:  0.27225462845549075 , 0.6370783667258433 , 0.9074308901851382 

sine:
Final Score:  0.25199086989601827 ± 0.023027921426126016



In [7]:
### Predictive score
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training: 100%|██████████| 5000/5000 [01:22<00:00, 60.29it/s] 


0  epoch:  0.2508223899922879 



training: 100%|██████████| 5000/5000 [01:23<00:00, 59.87it/s]


1  epoch:  0.2509466521846742 



training: 100%|██████████| 5000/5000 [01:18<00:00, 63.72it/s]


2  epoch:  0.250940877047357 



training: 100%|██████████| 5000/5000 [01:13<00:00, 67.86it/s] 


3  epoch:  0.25096965289993906 



training: 100%|██████████| 5000/5000 [01:15<00:00, 66.32it/s] 


4  epoch:  0.25112682312418216 

sine:
Final Score:  0.25096127904968807 ± 0.000135158019462929

